In [23]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import plotly.graph_objects as go
import plotly.express as px


In [24]:
airline_data=pd.read_csv("airline_data.csv")

In [25]:
airline_data.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
year_option=[]
for x in range(2005,2021):
    year_option.append({"label":str(x),"value":str(x)})
year_option

[{'label': '2005', 'value': '2005'},
 {'label': '2006', 'value': '2006'},
 {'label': '2007', 'value': '2007'},
 {'label': '2008', 'value': '2008'},
 {'label': '2009', 'value': '2009'},
 {'label': '2010', 'value': '2010'},
 {'label': '2011', 'value': '2011'},
 {'label': '2012', 'value': '2012'},
 {'label': '2013', 'value': '2013'},
 {'label': '2014', 'value': '2014'},
 {'label': '2015', 'value': '2015'},
 {'label': '2016', 'value': '2016'},
 {'label': '2017', 'value': '2017'},
 {'label': '2018', 'value': '2018'},
 {'label': '2019', 'value': '2019'},
 {'label': '2020', 'value': '2020'}]

In [27]:
def data_choice_1(df):
    bar_data=df.groupby(["Month","CancellationCode"])["Flights"].sum().reset_index()
    line_data=df.groupby(["Month","Reporting_Airline"])["AirTime"].mean().reset_index()
    div_data=df[df["DivAirportLandings"] !=0.0]
    map_data=df.groupby(["OriginState"])["Flights"].sum().reset_index()
    tree_data=df.groupby(["DestState","Reporting_Airline"])["Flights"].sum().reset_index()
    
    return bar_data, line_data, div_data, map_data, tree_data
    

In [28]:
def data_choice_2(df):
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late
    

In [37]:
app=dash.Dash(__name__)

In [38]:
app.layout=html.Div(children=[
    html.H1("US Domestic Airline Flights Performance",
            style={"height":"45px","textAlign":"center","color":"#f5f0e1","font-size":30,"backgroundColor":"#031163"}),
    html.Div(children=[
            html.Div("Option:",style={"color":"#031163","font-size":20}),
             dcc.Dropdown(id="input-type",options=[
                 {"label":"Yearly Airline Performance Report","value":"OPT1"},
                 {"label":"Yearly Airline Delay Report","value":"OPT2"}
             ],value='OPT1',placeholder="Select report type",style={"color":"#031163","marginLeft":"2px","width":"60%"})],
                style={"width":"100%","display":"flex"}),
   html.Br(),
   html.Div(children=[
             html.Div("Year:",style={"color":"#031163","font-size":20}),
             dcc.Dropdown(id="year",options=year_option,value=2010,
             placeholder="Select year",style={"marginLeft":"2px","width":"50%"})],
                 style={"color":"#031163","width":"100%","display":"flex"}),
    html.Hr(style={"color":"#031163","height":"3px"}),
    html.Div(dcc.Graph(id="gr1"),style={"border":"2px solid #031163"}),
   html.Div(children=[
             html.Div(dcc.Graph(id="gr2"),style={"border":"2px solid #031163"}),
             html.Div(dcc.Graph(id="gr3"),style={"border":"2px solid #031163"})],style={"color":"#031163","display":"flex"}),
    html.Div(children=[
            html.Div(dcc.Graph(id="gr4"),style={"border":"2px solid #031163"}),
             html.Div(dcc.Graph(id="gr5"),style={"border":"2px solid #031163"})],style={"color":"#031163","display":"flex"}),
    
    
],style={"width":"100%","backgroundColor":" #f5f0e1"})


In [39]:
@app.callback([
Output("gr1","figure"),
Output("gr2","figure"),
Output("gr3","figure"),
Output("gr4","figure"),
Output("gr5","figure")],
Input("input-type","value"),
Input("year","value"))

def data(chart,eyear):
    df=airline_data[airline_data["Year"]==int(eyear)]
    if chart=="OPT1":
        bar_data, line_data, div_data, map_data, tree_data=data_choice_1(df)
        bar_fig=px.bar(bar_data,x="Month",y="Flights",color="CancellationCode",title="Monthly Flight Cancellation")
            
        line_fig=px.line(line_data,x="Month",y="AirTime",color="Reporting_Airline",title="Avg. Monthly flight time (Minutes) by Airline")
            
        pie_fig=px.pie(div_data,values="Flights",names="Reporting_Airline",title="% of flights by reporting airline in year "+str(eyear))
            
        map_fig=px.choropleth(map_data,locations="OriginState",color="Flights",hover_data=["OriginState","Flights"],
                                 locationmode="USA-states",
                                 color_continuous_scale="GnBu",
                                 range_color=[0, map_data['Flights'].max()])
        map_fig.update_layout(title_text="Number of Flights From Origin State",geo_scope="usa")
            
        tree_fig=px.treemap(tree_data,path=["DestState","Reporting_Airline"],values="Flights",color="Flights",color_continuous_scale="RdBu"
                               ,title="Flight count by airline to destination State")
        
        tree_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        bar_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        line_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        pie_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        map_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        return tree_fig,pie_fig,map_fig,bar_fig,line_fig
    else:
        avg_car, avg_weather, avg_NAS, avg_sec, avg_late=data_choice_2(df)
        
        carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
        
        weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
        
        nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
        
        sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
        
        late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
        
        carrier_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        weather_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        nas_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        sec_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        late_fig.update_layout(paper_bgcolor="#f5f0e1",plot_bgcolor="#f5f0e1")
        
        return carrier_fig,weather_fig,nas_fig,sec_fig,late_fig
  

   
   
        
            
            
           
         

In [ ]:
if __name__=="__main__":
    app.run_server(host="localhost")

Dash is running on http://localhost:8050/

Dash is running on http://localhost:8050/

Dash is running on http://localhost:8050/

Dash is running on http://localhost:8050/

Dash is running on http://localhost:8050/

Dash is running on http://localhost:8050/

Dash is running on http://localhost:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jun/2021 11:19:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2021 11:19:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2021 11:19:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2021 11:19:29] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
127.0.0.1 - - [15/Jun/2021 11:19:30] "POST /_dash-update-component HTTP/1.1" 200 -
